In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition
import matplotlib.pyplot as plt
import numpy as np
import re
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
df = pd.read_csv('https://github.com/srivatsan88/YouTubeLI/blob/master/dataset/consumer_compliants.zip?raw=true', compression='zip', sep=',', quotechar='"')

In [5]:
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,4/3/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Fraudulent loan,This auto loan was opened on XX/XX/2020 in XXX...,Company has responded to the consumer and the ...,TRUIST FINANCIAL CORPORATION,PA,None,None,Consent provided,Web,4/3/2020,Closed with explanation,Yes,NaN,3591341
1,3/12/2020,Debt collection,Payday loan debt,Attempts to collect debt not owed,Debt is not yours,In XXXX of 2019 I noticed a debt for {$620.00}...,None,CURO Intermediate Holdings,CO,806XX,None,Consent provided,Web,3/12/2020,Closed with explanation,Yes,NaN,3564184
2,2/6/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Credit denial,"As stated from Capital One, XXXX XX/XX/XXXX an...",None,CAPITAL ONE FINANCIAL CORPORATION,OH,430XX,None,Consent provided,Web,2/6/2020,Closed with explanation,Yes,NaN,3521949
3,3/6/2020,Checking or savings account,Savings account,Managing an account,Banking errors,"Please see CFPB case XXXX. \n\nCapital One, in...",None,CAPITAL ONE FINANCIAL CORPORATION,CA,None,None,Consent provided,Web,3/6/2020,Closed with explanation,Yes,NaN,3556237
4,2/14/2020,Debt collection,Medical debt,Attempts to collect debt not owed,Debt is not yours,This debt was incurred due to medical malpract...,Company believes it acted appropriately as aut...,"Merchants and Professional Bureau, Inc.",OH,432XX,None,Consent provided,Web,2/14/2020,Closed with explanation,Yes,NaN,3531704


In [6]:
df['Product'].value_counts()

Debt collection                21772
Credit card or prepaid card    13193
Mortgage                        9799
Checking or savings account     7003
Student loan                    2950
Vehicle loan or lease           2736
Name: Product, dtype: int64

In [8]:
df['Company'].value_counts()

CITIBANK, N.A.                           3226
CAPITAL ONE FINANCIAL CORPORATION        2711
BANK OF AMERICA, NATIONAL ASSOCIATION    2580
JPMORGAN CHASE & CO.                     2409
WELLS FARGO & COMPANY                    2001
                                         ... 
Law Offices of Jeffrey Nadel                1
Med-1 Solutions, LLC                        1
Fradkin & Weber PA                          1
ClosingMark Financial Services, LLC         1
Wellmont Health System                      1
Name: Company, Length: 2197, dtype: int64

In [22]:
complaints_df = df[['Consumer complaint narrative', 'Product', 'Company']]
complaints_df = complaints_df.rename(columns={'Consumer complaint narrative': 'complaints'})

In [23]:
pd.set_option('display.max_colwidth', -1)
complaints_df.columns

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


Index(['complaints', 'Product', 'Company'], dtype='object')

In [24]:
x_train, x_hold = train_test_split(complaints_df, test_size=0.6, random_state=111)

In [25]:
x_train.head(5)

,complaints,Product,Company
36524,"On XX/XX/2020. Checks for the full amounts owed were sent to each of these accounts from XXXX to Capital One : Capital One ending in # XXXX {$3700.00} Capital One ending in # XXXX {$680.00} Capital One ending in # XXXX {$2600.00} Capital One ending in # XXXX {$2100.00} Capital One ending in # XXXX {$3400.00} Total payment on these accounts : {$12000.00}. \nPrior to these payoff checks sent, I made my monthly payments due XX/XX/XXXX to avoid any late fees while these checks were received and processed by Capital One. \nI later received statements showing I owed interest on each one after payment in full was received. After sending them a letter priority mail on XX/XX/2020, asking why I was paying interest on top of interest, I received a letter on XX/XX/2020 letting me know the accounts were closed per my request, which I had requested they close since I was paying in full BUT no answer to my question. I attempted to call several times prior to mailing this letter but had not success in getting anyone on the line after waiting over an hour. I went ahead and paid the interest on these accounts which added to {$280.00}, out of fear I may be hit with a late fee. I attempted to call again XX/XX/2020. After waiting over an hour for a rep on phone, I was told I had to pay the interest on interest and also a residual fee after for each one. ( how do you pay interest on interest? ) Residual fee? On accounts paid in full? They said, that was their policy. \nShortly after making full payments on these accounts, On XX/XX/XXXX, my husband lost his job due to COVID and is currently unemployed. I have struggled to pay these additional fees that I find to be an illegal act by this credit card company. I was told if I did not pay the additional fees, a late fee will be added on top. I should not have to pay interest or residual fees on a payoff amount that had not outstanding balance. I also requested a copy of their agreement stating they are allowed to do this. I never received a response or any response to any of my questions. Again, we are in no financial position at this time due to loss of unemployment to be slammed with extra fees. It is unfortunate that the timing on these payments made in full and my husband losing his job came at a worse time.",Credit card or prepaid card,CAPITAL ONE FINANCIAL CORPORATION
46281,"On or about XX/XX/XXXX I opened a Citibank Basic Checking Account after seeing an offer described by Citibank that if I opened a Basic Checking Account, deposited at least {$5000.00} within 30 days of accounting opening, and held at least that amount in the account for 60 days that within 90 days after fulfilling the conditions of the agreement I would receive a {$200.00} checking account bonus. I soon deposited approximately {$5000.00} on XX/XX/XXXX into the account. On XX/XX/XXXX I received another email update from Citibank about the offer and thanking me for enrolling in the Citibank {$200.00} checking offer and repeating the terms of the agreement. \n\nOn or about XX/XX/XXXX I utilized the Citibank Mobile Application 's chat function to inquire as to when my bonus might be depositing, as it had been well over 60 days since when I opened the account and deposited the qualifying funds on XX/XX/XXXX. 60 days since my deposit would have been around early XX/XX/XXXX. The Mobile Application representative stated they could not resolve the issue and to call their One Stop Sales Unit, which I did on XX/XX/XXXX. The representative at the One Stop Sales Unit was confused and did not know how to search for the offer clearly, saying too that often the offers are for holding money in the account for 90 days or more, and provided essentially no clarity or guidance. \n\nI withdrew {$3000.00} of the {$5000.00} I had held in the account since XX/XX/XXXX on XX/XX/XXXX, my first withdrawl from the account. I am concerned as Citibank has been very unclear on when my deposit bonus will be depositing, as tha

In [26]:
stemmer = PorterStemmer()

In [27]:
def tokenize(text):
  tokens = [word for word in nltk.word_tokenize(text) if(len(word) > 3 and len(word.strip('Xx/')) > 2)]
  return tokens

In [28]:
vectorizer_df = TfidfVectorizer(tokenizer=tokenize,
                                stop_words = 'english',
                                max_df=0.75, min_df=50, max_features=10000,
                                use_idf=False, norm=None)

tf_vectors = vectorizer_df.fit_transform(x_train.complaints)

In [31]:
tf_vectors.A

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [32]:
vectorizer_df.get_feature_names()

['0.00',
 '1.00',
 '10.00',
 '100.00',
 '1000.00',
 '10000.00',
 '100000.00',
 '110.00',
 '1100.00',
 '11000.00',
 '12.00',
 '120.00',
 '1200.00',
 '12000.00',
 '130.00',
 '1300.00',
 '13000.00',
 '140.00',
 '1400.00',
 '14000.00',
 '15.00',
 '150.00',
 '1500.00',
 '15000.00',
 '160.00',
 '1600.00',
 '16000.00',
 '1681c-2',
 '1692',
 '1692g',
 '170.00',
 '1700.00',
 '180.00',
 '1800.00',
 '18000.00',
 '190.00',
 '1900.00',
 '2.00',
 '20.00',
 '200.00',
 '2000.00',
 '20000.00',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2019.',
 '2020',
 '210.00',
 '2100.00',
 '220.00',
 '2200.00',
 '230.00',
 '2300.00',
 '240.00',
 '2400.00',
 '25.00',
 '250.00',
 '2500.00',
 '25000.00',
 '260.00',
 '2600.00',
 '27.00',
 '270.00',
 '2700.00',
 '28.00',
 '280.00',
 '2800.00',
 '29.00',
 '290.00',
 '2900.00',
 '3.00',
 '30.00',
 '300.00',
 '3000.00',
 '30000.00',
 '310.00',
 '320.00',
 '3200.00',
 '3300.00',
 '340.00',
 '3400.00',
 '35.00',
 '350.00',
 '3500.00',
 '36.00',
 '360.00',
 '3600.00',
 '37

In [35]:
# n_components: no. of topics
# max_iter: more the iterations, better the performance


lda = decomposition.LatentDirichletAllocation(n_components=6, max_iter=3, learning_method='online', learning_offset=50,n_jobs=1, random_state=111)
W1 = lda.fit_transform(tf_vectors)
H1 = lda.components_

In [36]:
W1

array([[0.39738388, 0.00119779, 0.22199229, 0.25544898, 0.00119199,
        0.12278506],
       [0.00126202, 0.06441092, 0.60474967, 0.32704835, 0.00126163,
        0.00126741],
       [0.12856772, 0.00559638, 0.20017757, 0.65447287, 0.00557809,
        0.00560737],
       ...,
       [0.09245675, 0.00304748, 0.17866742, 0.71969463, 0.00306321,
        0.00307051],
       [0.01686614, 0.01675331, 0.73817173, 0.19464288, 0.01681425,
        0.01675169],
       [0.19913084, 0.66215434, 0.00137137, 0.00137076, 0.13459089,
        0.0013818 ]])

In [37]:
num_words=15
vocab = np.array(vectorizer_df.get_feature_names())

top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_words-1:-1]]
topic_words = ([top_words(t) for t in H1])
topics = [' '.join(t) for t in topic_words]

In [38]:
topics

['payment paid account balance payments fees insurance late statement received month charged company make date',
 'loan mortgage home received property letter sent documents company modification time process servicing closing request',
 'card credit account charges dispute capital charge received citi bank closed fraud xx/xx/2019 report citibank',
 'account bank told called said phone money number check asked received time chase funds email',
 'debt credit collection report account company information reporting letter sent agency provide received original identity',
 'payment payments credit time told years loan loans month make months help called late just']

In [42]:
columns = ["Topic" + str(i) for i in range(lda.n_components)]
docnames = ["Doc" + str(i) for i in range(len(x_train.complaints))]
df_doc_topic = pd.DataFrame(np.round(W1,2), columns=columns, index=docnames)
significant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['dominant_topic'] = significant_topic

In [43]:
df_doc_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant_topic
Doc0,0.40,0.00,0.22,0.26,0.00,0.12,0
Doc1,0.00,0.06,0.60,0.33,0.00,0.00,2
Doc2,0.13,0.01,0.20,0.65,0.01,0.01,3
Doc3,0.00,0.00,0.00,0.42,0.57,0.00,4
Doc4,0.00,0.00,0.00,0.19,0.79,0.00,4
...,...,...,...,...,...,...,...
Doc22976,0.00,0.00,0.71,0.15,0.00,0.13,2
Doc22977,0.00,0.06,0.17,0.29,0.17,0.31,5
Doc22978,0.09,0.00,0.18,0.72,0.00,0.00,3
Doc22979,0.02,0.02,0.74,0.19,0.02,0.02,2


In [44]:
Whold = lda.transform(vectorizer_df.transform(x_hold.complaints[:5]))

In [47]:
columns = ["Topic" + str(i) for i in range(lda.n_components)]
docnames = ["Doc" + str(i) for i in range(len(x_hold.complaints[:5]))]
df_doc_topic = pd.DataFrame(np.round(Whold,2), columns=columns, index=docnames)
significant_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['dominant_topic'] = significant_topic

In [48]:
df_doc_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,dominant_topic
Doc0,0.12,0.00,0.07,0.80,0.00,0.00,3
Doc1,0.21,0.63,0.00,0.15,0.00,0.00,1
Doc2,0.21,0.01,0.42,0.36,0.01,0.01,2
Doc3,0.01,0.22,0.00,0.00,0.00,0.76,5
Doc4,0.30,0.00,0.19,0.11,0.00,0.39,5
